In [19]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np
from collections import Counter

#df = pd.read_csv('data/data-from-emma/SmarterStayAlive.csv')
df = pd.read_csv('data/data-from-emma/Target.csv')

print(df.columns)

print(df.shape)

print(Counter(df.seed))



Index(['agent_name', 'seed', 't', 'action', 'missed_ball', 'xpos_ball',
       'ypos_ball', 'xpos_ball_prev', 'ypos_ball_prev', 'xpos_pad', 'ypos_pad',
       'xpos_pad_prev', 'ypos_pad_prev', 'board_alive', 'is_far_left',
       'is_far_right', 'score', 'pad_width', 'ball_speed', 'ball_down',
       'xdist_ball_pad', 'ydist_ball_pad', 'l2_ball_pad', 'num_bricks_left'],
      dtype='object')
(38241, 24)
Counter({9654690: 2000, 127525: 2000, 6816947: 2000, 6169021: 2000, 5509973: 2000, 596737: 2000, 9334909: 2000, 122168: 1816, 9989550: 1802, 3648516: 1752, 1116028: 1743, 3226525: 1702, 2247560: 1595, 4561254: 1588, 3736793: 1488, 9261983: 1338, 2241640: 1162, 9012387: 1067, 5082548: 1052, 3681492: 924, 7670649: 875, 2091658: 789, 6759291: 696, 1001231: 679, 6782754: 533, 3451451: 484, 693342: 401, 4045096: 323, 2401381: 293, 9583670: 139})


### The dataframe contains time steps for 30 different seeds


In [2]:
print(len(np.unique(df['seed'])))

30


In [20]:
df[pd.isnull(df['action'])]

,agent_name,seed,t,action,missed_ball,xpos_ball,ypos_ball,xpos_ball_prev,ypos_ball_prev,xpos_pad,...,is_far_left,is_far_right,score,pad_width,ball_speed,ball_down,xdist_ball_pad,ydist_ball_pad,l2_ball_pad,num_bricks_left
1996,Target,9654690,1997,NaN,False,210.053265,74.397636,208.426318,70.743454,216.0,...,False,False,394,small,fast,True,5.946735,68.602364,68.859625,26
1997,Target,9654690,1998,NaN,False,211.680211,78.051818,210.053265,74.397636,216.0,...,False,False,394,small,fast,True,4.319789,64.948182,65.091681,26
1998,Target,9654690,1999,NaN,False,213.307158,81.706000,211.680211,78.051818,220.0,...,False,False,394,small,fast,True,6.692842,61.294000,61.658321,26
1999,Target,9654690,2000,NaN,False,214.934104,85.360182,213.307158,81.706000,216.0,...,False,False,394,small,fast,True,1.065896,57.639818,57.649673,26
3996,Target,127525,1997,NaN,False,177.131563,132.188023,175.504617,135.842205,168.0,...,False,False,379,small,fast,False,9.131563,10.811977,14.152183,29
3997,Target,127525,1998,NaN,False,178.758510,128.533841,177.131563,132.188023,172.0,...,False,False,379,small,fast,False,6.758510,14.466159,15.967066,29
3998,Target,127525,1999,NaN,False,180.385456,124.879659,178.758510,128.533841,176.0,...,False,False,379,small,fast,False,4.385456,18.120341,18.643470,29
3999,Target,127525,2000,NaN,False,182.012403,121.225478,180.385456,124.879659,180.0,...,False,False,379,small,fast,False,2.012403,21.774522,21.867318,29
5996,Target,6816947,1997,NaN,False,91.560595,93.204562,94.533175,90.528040,112.0,...,False,False,134,big,fast,True,20.439405,49.795438,53.827083,61
5997,Target,6816947,1998,NaN,False,88.588016,95.881085,91.560595,93.204562,108.0,...,False,False,134,big,fast,True,19.411984,47.118915,50.960939,61


#### Initially, we'll say "why did you go left?" (instead of 'right' or 'button1') is the outcome/query of interest

## Steps for training a model
- Calculate the outcome variable at every time point
- looping over all episodes, sample positive and negative data points
- discretize all variables

### We first have to do some variable construction, specifically from the board_alive variable

- First convert to brick counts by column

In [3]:
#len(np.unique(df['board_alive']))
#for new_col in new_col_names:
#    df[new_col] = ""
new_col_data = []
new_col_names = ["bricks_in_col_" + str(col).zfill(2) for col in range(0,18)]

df = df.reindex(columns=df.columns.tolist() + new_col_names)#add new (empty) columns to the dataframe

for row in range(0,len(df.index)):
    board_alive = bin(int(df['board_alive'][row]))[2:].zfill(108)
    cols = [board_alive[(i*6):(i*6+6)] for i in range(0,18)]#pull out each column from the binary string (every 7 digits)
    col_counts = [sum([int(val) for val in list(col)]) for col in cols]#convert each string to a list of ints and sum them
    new_col_data.append(col_counts)

df[new_col_names] = new_col_data


In [4]:
df.iloc[3400]

agent_name                                    Target
seed                                          127525
t                                               1401
action                                       button1
missed_ball                                    False
xpos_ball                                    81.1417
ypos_ball                                    33.5251
xpos_ball_prev                               82.7687
ypos_ball_prev                               29.8709
xpos_pad                                          80
ypos_pad                                         143
xpos_pad_prev                                     84
ypos_pad_prev                                    143
board_alive         60877557880230637991369707094015
is_far_left                                    False
is_far_right                                   False
score                                            128
pad_width                                      small
ball_speed                                    

### Create variables from the column counts
- number of columns with 0, 1, or 2 bricks remaining
- number of columns left of the paddle w/1 or 2 bricks remaining
- number of bricks remaining left of the paddle
- number of bricks remaining right of the paddle

In [5]:
df['num_cols_0_bricks'] = [sum(df.iloc[row][new_col_names] == 0) for row in range(0,len(df.index))]

In [6]:
df['num_cols_1_brick'] = [sum(df.iloc[row][new_col_names] == 1) for row in range(0,len(df.index))]

In [7]:
df['num_cols_2_bricks'] = [sum(df.iloc[row][new_col_names] == 2) for row in range(0,len(df.index))]

In [8]:
#hard-coded column positions, taken from the json of a random Breakout game
col_positions = [i*12 for i in range(1,19)]

bricks_left_of_paddle = []
bricks_right_of_paddle = []
channels_left_of_paddle = []
channels_right_of_paddle = []
almost_channels_left_of_paddle = []
almost_channels_right_of_paddle = []

for row in range(0, len(df.index)):
    paddle_pos = df.iloc[row]['xpos_pad']
    cols_left_of_paddle = np.where(col_positions < paddle_pos)[0]#get which columns are left of paddle
    cols_left_of_paddle = [str(i).zfill(2) for i in cols_left_of_paddle]#convert into a padded string to match column names
    cols_right_of_paddle = np.where(col_positions > paddle_pos)[0]#get which columns are left of paddle
    cols_right_of_paddle = [str(i).zfill(2) for i in cols_right_of_paddle]#convert into a padded string to match column names
 
    prefix = new_col_names[0][0:(len(new_col_names[0])-2)]
    cols_left_of_paddle = [prefix + col for col in cols_left_of_paddle]#add the prefix to match column names
    cols_right_of_paddle = [prefix + col for col in cols_right_of_paddle]#add the prefix to match column names

    vals_left_of_paddle = df.iloc[row][cols_left_of_paddle]
    vals_right_of_paddle = df.iloc[row][cols_right_of_paddle]
    bricks_left_of_paddle.append(sum(vals_left_of_paddle))
    bricks_right_of_paddle.append(sum(vals_right_of_paddle))
    
    channels_left_of_paddle.append(sum(vals_left_of_paddle == 0))
    channels_right_of_paddle.append(sum(vals_right_of_paddle == 0))

    almost_channels_left_of_paddle.append(sum([(val in [1,2]) for val in vals_left_of_paddle]))
    almost_channels_right_of_paddle.append(sum([(val in [1,2]) for val in vals_right_of_paddle]))


df['bricks_left_of_paddle'] = bricks_left_of_paddle
df['bricks_right_of_paddle'] = bricks_right_of_paddle
df['channels_left_of_paddle'] = channels_left_of_paddle
df['channels_right_of_paddle'] = channels_right_of_paddle
df['almost_channels_left_of_paddle'] = almost_channels_left_of_paddle
df['almost_channels_right_of_paddle'] = almost_channels_right_of_paddle

In [9]:
df.iloc[3400]

agent_name                                                   Target
seed                                                         127525
t                                                              1401
action                                                      button1
missed_ball                                                   False
xpos_ball                                                   81.1417
ypos_ball                                                   33.5251
xpos_ball_prev                                              82.7687
ypos_ball_prev                                              29.8709
xpos_pad                                                         80
ypos_pad                                                        143
xpos_pad_prev                                                    84
ypos_pad_prev                                                   143
board_alive                        60877557880230637991369707094015
is_far_left                                     

In [17]:
predictors = ['missed_ball', 'xpos_ball', 'ypos_ball', 'xpos_ball_prev', 'ypos_ball_prev', 'xpos_pad',
        'xpos_pad_prev', 'is_far_left', 'is_far_right', 'score', 'pad_width', 'ball_speed', 'ball_down',
        'xdist_ball_pad', 'ydist_ball_pad', 'l2_ball_pad', 'num_bricks_left', 'num_cols_0_bricks', 'num_cols_1_brick',
        'num_cols_2_bricks', 'bricks_right_of_paddle', 'bricks_left_of_paddle', 'channels_left_of_paddle',
        'channels_right_of_paddle', 'almost_channels_left_of_paddle', 'almost_channels_right_of_paddle']

## We have to do some data cleaning - some predictors have NaN fields
- remove any rows that have no action
- for xpos_ball_prev and ypos_ball_prev at t=0, set to xpos_ball and ypos_ball
- same for xpos_pad_rev and ypos_pad_prev

In [12]:
has_action = [not pd.isnull(val) for val in df['action']]
df = df[has_action]

nans = np.isnan(df['xpos_ball_prev'])
df.loc[nans,'xpos_ball_prev'] = df.loc[nans, 'xpos_ball']
df.loc[nans,'ypos_ball_prev'] = df.loc[nans, 'ypos_ball']

nans = np.isnan(df['xpos_pad_prev'])
df.loc[nans,'xpos_pad_prev'] = df.loc[nans, 'xpos_pad']
df.loc[nans,'ypos_pad_prev'] = df.loc[nans, 'ypos_pad']


#df['pad_width'] = [1 if val == 'big' else 0 for val in df['pad_width']]

#df['ball_speed'] = [1 if val == 'fast' else 0 for val in df['ball_speed']]

#### Discretize all predictors (outcome is already boolean)
#### Convert booleans to 0,1

In [13]:
for predictor in predictors:
    print(predictor)
    if isinstance(df[predictor][0], (np.bool_, bool)):
        print(predictor + " is boolean")
        #convert booleans to 0/1 rather than 'True'/'False'
        df[predictor] = [1 if val == 'True' else 0 for val in df[predictor]]
    if isinstance(df[predictor][0], str):
        if len(np.unique(df[predictor])) > 2:
            print("warning: " + predictor + " should be discretized")
        else:#don't discretize, just convert to integer values, since order is irrelevant for booleans
            df[predictor] = [1 if val == df[predictor][0] else 0 for val in df[predictor]]
            
    

missed_ball
missed_ball is boolean
xpos_ball
ypos_ball
xpos_ball_prev
ypos_ball_prev
xpos_pad
xpos_pad_prev
is_far_left
is_far_left is boolean
is_far_right
is_far_right is boolean
score
pad_width
ball_speed
ball_down
ball_down is boolean
xdist_ball_pad
ydist_ball_pad
l2_ball_pad
num_bricks_left
num_cols_0_bricks
num_cols_1_brick
num_cols_2_bricks
bricks_right_of_paddle
bricks_left_of_paddle
channels_left_of_paddle
channels_right_of_paddle
almost_channels_left_of_paddle
almost_channels_right_of_paddle


#### For each seed, get all positive data points and sample an equal number of negative data points

In [14]:
df['outcome'] = (df['action'] == 'left')

for seed in np.unique(df['seed']):
    positive_samples = df[(df['seed'] == seed) & df.outcome]
    negative_samples = df[(df['seed'] == seed) & ~df.outcome]
    negative_samples = negative_samples.sample(n=len(positive_samples.index))
samples = pd.concat([positive_samples, negative_samples])

print(positive_samples.shape)
print(negative_samples.shape)
print(samples.shape)


(491, 52)
(491, 52)
(982, 52)


In [18]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

rf.fit(samples[predictors], samples['outcome'])

rf.predict(df[predictors])

array([0.407, 0.407, 0.428, ..., 0.507, 0.514, 0.536])